<a href="https://colab.research.google.com/github/TheClassicTechno/cleansea_model/blob/main/another_version/small_taco_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pathlib
import cv2
import glob

#processing images
bottles1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/bottles'
cans1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/cans'
containers1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/containers'
#paper1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/paper'
plastic1 = '/content/drive/MyDrive/TEAM9SURESTART/taco/plastic'

#all images are 227x227 in RGB so 227, 227, 3
bottles = [cv2.imread(image) for image in glob.glob(bottles1)]
cans = [cv2.imread(image) for image in glob.glob(cans1)]
containers = [cv2.imread(image) for image in glob.glob(containers1)]
#paper = [cv2.imread(image) for image in glob.glob(paper1)]
plastic = [cv2.imread(image) for image in glob.glob(plastic1)]

train_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/TEAM9SURESTART/taco',
    validation_split = 0.2,
    subset = "training",
    seed = 24,  
    image_size = (256, 256),
    batch_size = 32
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/TEAM9SURESTART/taco',
    validation_split = 0.2,
    subset = "validation",
    seed = 24,  
    image_size = (256, 256),
    batch_size = 32
)
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(train_dataset, val_dataset, test_size = 0.1, random_state = 1, stratify = val_dataset)

from scipy.stats import zscore
import pandas as pd

# Create a sample df

# Calculate the zscores and drop zscores into new column

train_dataset = train_dataset.cache().shuffle(13).prefetch(buffer_size = tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().shuffle(13).prefetch(buffer_size = tf.data.AUTOTUNE)

model = Sequential ([
    layers.Rescaling(1./255, input_shape = (256, 256, 3)),
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Conv2D(32, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Conv2D(16, 3, padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(4, activation = 'softmax')
])
model.summary()


Found 163 files belonging to 4 classes.
Using 131 files for training.
Found 163 files belonging to 4 classes.
Using 32 files for validation.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 128, 128, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                               

In [3]:
model.compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])
'''
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy'],  run_eagerly=True
)
'''

model.fit (
    train_dataset, 
    validation_data = val_dataset,
    epochs = 250
)
#model.save('model.h5')

Epoch 1/250


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 17s 299ms/step - loss: 2.7161 - accuracy: 0.2519 - val_loss: 1.3580 - val_accuracy: 0.2812
Epoch 2/250
5/5 [==============================] - 0s 26ms/step - loss: 1.4398 - accuracy: 0.3664 - val_loss: 1.3706 - val_accuracy: 0.5000
Epoch 3/250
5/5 [==============================] - 0s 25ms/step - loss: 1.3710 - accuracy: 0.3130 - val_loss: 1.3836 - val_accuracy: 0.3750
Epoch 4/250
5/5 [==============================] - 0s 25ms/step - loss: 1.3786 - accuracy: 0.3130 - val_loss: 1.3859 - val_accuracy: 0.0938
Epoch 5/250
5/5 [==============================] - 0s 26ms/step - loss: 1.3801 - accuracy: 0.3282 - val_loss: 1.3853 - val_accuracy: 0.2812
Epoch 6/250
5/5 [==============================] - 0s 25ms/step - loss: 1.3773 - accuracy: 0.3511 - val_loss: 1.3841 - val_accuracy: 0.2812
Epoch 7/250
5/5 [==============================] - 0s 26ms/step - loss: 1.3753 - accuracy: 0.4046 - val_loss: 1.3824 - val_accuracy: 0.2812
Epoch 8/250
5/5 [=============

KeyboardInterrupt: ignored

In [ ]:
loss = pd.DataFrame(model.history.history)

#plotting the loss and accuracy 
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.plot(loss["loss"], label ="Loss")
plt.plot(loss["val_loss"], label = "Validation_loss")
plt.legend()
plt.title("Training and Validation Loss")

plt.subplot(2,2,2)
plt.plot(loss['accuracy'],label = "Training Accuracy")
plt.plot(loss['val_accuracy'], label ="Validation_ Accuracy ")
plt.legend()
plt.title("Training-Validation Accuracy")